<a href="https://colab.research.google.com/github/dpostolovski/eeg_is/blob/train_compare_full_data/newest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#@title Инсталирање и вчитување на потребните библиотеки

# Библиотека за истражување, визуелизација и анализирање на човечки 
# неврофизиолошки податоци (EEG, sEEG и др)
!pip install mne 
!pip install termcolor
!wget "https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py"
!mkdir saved_models

%tensorflow_version 1.12.0

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
from termcolor import colored
import mne
from sklearn.decomposition import PCA, FastICA
import mne
from sklearn import metrics 
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint 
from sklearn.model_selection import train_test_split
from EEGModels import EEGNet,ShallowConvNet
import scipy.io as sio
from scipy.fft import fft
from scipy import signal
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras import backend as K
from tensorflow import keras
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from EEGModels import EEGNet,ShallowConvNet,DeepConvNet
from tensorflow.keras.optimizers import Adam

K.set_image_data_format('channels_first')

     |████████████████████████████████| 6.6MB 1.6MB/s 
--2020-05-24 21:30:57--  https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18283 (18K) [text/plain]
Saving to: ‘EEGModels.py’

EEGModels.py        100%[===================>]  17.85K  --.-KB/s    in 0.001s  

2020-05-24 21:30:57 (28.0 MB/s) - ‘EEGModels.py’ saved [18283/18283]

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.12.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [0]:
# Иницијализација на променливите каде ќе бидат вчитани податоците
data = []
labels = []
events = []
targets = []

for i in range(1, 16): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') + '-Train'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), 2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")
      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.003))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 1 се вчитани.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3200/3600 [=========================>....] - ETA: 0s - loss: 2.3914 - acc: 0.1644
Epoch 00001: val_loss improved from inf to 2.17458, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 8s 2ms/sample - loss: 2.3801 - acc: 0.1653 - val_loss: 2.1746 - val_acc: 0.1208
Epoch 2/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.1942 - acc: 0.1867
Epoch 00002: val_loss improved from 2.17458 to 2.13627, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 1s 188us/sample - loss: 2.1867 - acc: 0.1867 - val_loss: 2.1363 - val_acc: 0.1792
Epoch 3/400
3300/3600 [======================